#### Keras Tuner- Decide Number of Hidden Layers And Neuron In Neural Network

In [2]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [13]:
df=pd.read_csv('Real_Combine.csv')

In [14]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [15]:
X=df.iloc[:,:-1] ## independent features
y=df.iloc[:,-1] ## dependent features

In [23]:
y

0       219.720833
1       182.187500
2       154.037500
3       223.208333
4       200.645833
           ...    
1088    288.416667
1089    256.833333
1090    169.000000
1091    186.041667
1092    185.583333
Name: PM 2.5, Length: 1093, dtype: float64

#### Hyperparameters
1. How many number of hidden layers we should have?
2. How many number of neurons we should have in hidden layers?
3. Learning Rate 

In [16]:
def build_model(hp):
    model = keras.Sequential() #Front and Back Propogation
    for i in range(hp.Int('num_layers', 2, 20)): # We can have 2 to 20 hidden layers #'Int' is for range of value
        model.add(layers.Dense(units=hp.Int('units_' + str(i), # In each hidden layer we can have 32 to 512 nodes in each layer
                                            min_value=32,
                                            max_value=512,
                                            step=32), #increase in value of nodes in each epoch
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear')) #Output layer only one node since its regression problem
    model.compile(                                    #Compiling code and adding learning rate choice for weight updation
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [17]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error', #Validation Mean absolute error obtained when model is tuned. We use that. 
    max_trials=5,  #For each try we try different combination of hidden layer.  
    executions_per_trial=3, #Within each trial, we do 3 trials with different no of neurons within each hidden layer. 
    directory='project',
    project_name='Air Quality Index')

INFO:tensorflow:Reloading Oracle from existing project project\Air Quality Index\oracle.json


In [18]:
tuner.search_space_summary()

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [36]:
X_train.shape,y_train.shape

((765, 8), (765,))

In [37]:
tuner.search(X_train, y_train,
             epochs=10,
             validation_data=(X_test, y_test))

ValueError: Please provide as model inputs either a single array or a list of arrays. You passed: inputs=         T    TM    Tm     SLP     H   VV     V    VM
1068  21.7  30.0  12.2  1014.5  52.0  1.1   6.3  16.5
181   13.6  20.4   7.5  1020.7  67.0  1.1  10.2  40.7
425   32.8  40.6  26.0  1005.8  45.0  1.9   7.2  11.1
1019  28.3  35.8  26.8  1003.9  89.0  1.6   4.3  11.1
9     11.9  18.9   6.3  1020.1  76.0  1.1   8.3  20.6
...    ...   ...   ...     ...   ...  ...   ...   ...
1033  31.0  35.0  26.8  1004.1  76.0  2.6   6.1  14.8
763   24.9  32.4  15.2  1015.0  57.0  1.9   1.7   9.4
835   34.1  37.2  29.0   998.4  61.0  2.6  11.5  22.2
559   12.3  19.0   8.9  1015.8  91.0  0.5   4.6   9.4
684   30.7  34.8  25.6  1004.5  66.0  2.6   3.3  11.1

[765 rows x 8 columns]

In [10]:
tuner.results_summary()